In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

import ast
import csv
import json
import matplotlib.pyplot as plt
import seaborn as sns
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Dropout, Lambda, Concatenate,Conv2D,GlobalAveragePooling2D,multiply, Add
from sklearn.metrics import cohen_kappa_score ,confusion_matrix,cohen_kappa_score,accuracy_score,f1_score

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)  
num_classes=4

# Preparing input image patch pairs for the model

In [ ]:
def parse_rgb_channel(channel_data):
    return np.array(ast.literal_eval(channel_data))
def process_input(samplesdf):
    '''
    samplesdf: input data with RGB and NIR channels exported from GEE.
    Process the input data from the CSV file and convert it into a samplesizex11X11X4 array suitable for the model.
    '''
    X = []
    for index, row in samplesdf.iterrows():
        r_channel = parse_rgb_channel(row['R'])/255
        g_channel = parse_rgb_channel(row['G'])/255
        b_channel = parse_rgb_channel(row['B'])/255
        nir_channel = parse_rgb_channel(row['N'])/127
        rgb_array = np.stack((r_channel, g_channel,b_channel,nir_channel), axis=-1)
        X.append(rgb_array)
    return X

def prepare_modelinput(samplespath):
    # 'r'D:\Project_wildfire\Map\data\traintestsamples.csv''
    samplesall=pd.read_csv(samplespath)
    train_samples=samplesall[samplesall['category'] == 'train']
    test2010_samples = samplesall[(samplesall['category'] == 'test') & (samplesall['year'] == 2010)]
    test2022_samples = samplesall[(samplesall['category'] == 'test') & (samplesall['year'] == 2022)]
    print(samplesall.groupby(['year','category'])['class'].value_counts())
    xtrain = np.array(process_input(train_samples))
    ytrain = train_samples['class'].values
    xval2010 = np.array(process_input(test2010_samples))
    yval2010= test2010_samples['class'].values
    xval2022 = np.array(process_input(test2022_samples))
    yval2022= test2022_samples['class'].values
    xvalall=np.concatenate([xval2010,xval2022],axis=0)
    ytrain = tf.keras.utils.to_categorical(ytrain, num_classes=num_classes)
    yval2010 = tf.keras.utils.to_categorical(yval2010, num_classes=num_classes)
    yval2022 = tf.keras.utils.to_categorical(yval2022, num_classes=num_classes)
    yvalall=np.concatenate([yval2010,yval2022],axis=0)
    df_train = pd.DataFrame({
        'features': list(xtrain),
        'labels': list(ytrain)
    })
    df_train_shuffled = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
    xtrain= np.array(df_train_shuffled['features'].tolist())
    ytrain = np.array(df_train_shuffled['labels'].tolist())
    return xtrain, ytrain, xval2010, yval2010, xval2022, yval2022,xvalall,yvalall


# Model 

## Model structure

In [ ]:
def spatial_attention(inputs, kernel_size=5):
    avg_pooling = Lambda(lambda x: K.mean(x, axis=-1, keepdims=True))(inputs)
    max_pooling = Lambda(lambda x: K.max(x, axis=-1, keepdims=True))(inputs)
    # print('max_pooling',max_pooling.shape)
    concat = Concatenate(axis=-1)([avg_pooling, max_pooling])
    cbam_feature = Conv2D(filters=1, kernel_size=(kernel_size,kernel_size), strides=1, padding='valid', activation='sigmoid',  use_bias=True)(concat)
    # print('cbam_feature',cbam_feature.shape)
    x=multiply([inputs, cbam_feature])
    return Add()([inputs,x])
def hycnn2d(inputshape,numers_filters,filter,drop,spat,bn):
    inputs= Input(shape=inputshape)
    x= tf.keras.layers.Conv2D(filters=numers_filters[0], kernel_size=(3, 3), padding='valid',use_bias=True)(inputs)
    if bn:
        x=tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ReLU()(x) # 9x9
    x = tf.keras.layers.Conv2D(filters=numers_filters[1], kernel_size=(3, 3), padding='valid',use_bias=True)(x)
    if bn:
        x=tf.keras.layers.BatchNormalization()(x) 

    x = tf.keras.layers.ReLU()(x) # 7x7
    x = tf.keras.layers.Conv2D(filters=numers_filters[2], kernel_size=(3, 3), padding='valid',use_bias=True)(x)
    if bn:
        x=tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ReLU()(x) # 5x5
    if spat:
        x=spatial_attention(x,kernel_size=5) # 5x5
    x = tf.keras.layers.Conv2D(filters=numers_filters[3], kernel_size=(3, 3), padding='valid',use_bias=True)(x) # 3x3
    if bn:
        x=tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters=numers_filters[4], kernel_size=(3, 3), padding='valid',use_bias=True)(x) # 1x1
    if bn:
        x=tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ReLU()(x)
    x=GlobalAveragePooling2D()(x)
    x = Dense(units=filter, activation='relu')(x)
    x = Dropout(drop)(x)
    output_layer = Dense(units=num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=output_layer)
    return model

## Visualizing and saving model metrics

In [ ]:
def plot_history(history,savejpgdir,savetitle):

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(savejpgdir, f'{savetitle}.jpg'))
    plt.close('all')

def confusionmatrix(model,saveconfusionjpgdir,year,savetitle,xval,yval):
    y_pred = model.predict(xval)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(yval, axis=1)

    cm = confusion_matrix(y_true, y_pred_classes)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=['Tree', 'Grass', 'Urban','Bare'],
                yticklabels=['Tree', 'Grass', 'Urban','Bare'])
    plt.xlabel('predict')
    plt.ylabel('true')
    plt.savefig(os.path.join(saveconfusionjpgdir, f'{year}{savetitle}.jpg'))
    plt.close('all')

def test_metrics(model,validation_data):
        val_predict = np.argmax(model.predict(validation_data[0],verbose=0), axis=-1)
        val_targ = np.argmax(validation_data[1], axis=-1)
        
        val_acc = accuracy_score(val_targ, val_predict)
        kappa = cohen_kappa_score(val_targ, val_predict)
        f1 = f1_score(val_targ, val_predict, average='macro')
        # confusion matrix
        cm = confusion_matrix(val_targ, val_predict)
        # calculate overall's producer accuracy and user accuracy
        producer_accuracy = np.diag(cm) / np.sum(cm, axis=1)
        user_accuracy = np.diag(cm) / np.sum(cm, axis=0)
        avg_producer_accuracy = np.mean(producer_accuracy)
        avg_user_accuracy = np.mean(user_accuracy)
        # calculate metrics for each class
        ptree = producer_accuracy[0] if len(producer_accuracy) > 1 else None
        utree = user_accuracy[0] if len(user_accuracy) > 1 else None
        f1tree = f1_score(val_targ, val_predict, labels=[0], average=None)[0] if 0 in val_targ else None

        pgrass = producer_accuracy[1] if len(producer_accuracy) > 1 else None
        ugrass = user_accuracy[1] if len(user_accuracy) > 1 else None
        f1grass = f1_score(val_targ, val_predict, labels=[1], average=None)[0] if 1 in val_targ else None

        phouse = producer_accuracy[2] if len(producer_accuracy) > 2 else None
        uhouse = user_accuracy[2] if len(user_accuracy) > 2 else None
        f1house = f1_score(val_targ, val_predict, labels=[2], average=None)[0] if 2 in val_targ else None
        
        pbare = producer_accuracy[3] if len(producer_accuracy) > 3 else None
        ubare = user_accuracy[3] if len(user_accuracy) > 3 else None
        f1bare = f1_score(val_targ, val_predict, labels=[3], average=None)[0] if 3 in val_targ else None

        return  val_acc,kappa,f1,avg_producer_accuracy,avg_user_accuracy,ptree,utree,f1tree,pgrass,ugrass,f1grass,phouse,uhouse,f1house,pbare,ubare,f1bare

## Model weights export for inferencing on GEE

In [ ]:
def fuse_conv_and_bn(conv_layer, bn_layer):
    """
    合并Conv2D层和BatchNormalization层的权重和偏置
    Args:
    conv_layer: Conv2D层
    bn_layer: BatchNormalization层
    Returns:
    fused_weights, fused_bias: 合并后的卷积层权重和偏置
    """

    # 获取卷积层权重和偏置
    conv_weights = conv_layer.get_weights()[0]  # W
    conv_bias = conv_layer.get_weights()[1] if len(conv_layer.get_weights()) > 1 else tf.zeros(conv_layer.filters)  # b

    # 获取BN层参数
    bn_gamma = bn_layer.gamma
    bn_beta = bn_layer.beta
    bn_moving_mean = bn_layer.moving_mean
    bn_moving_variance = bn_layer.moving_variance
    bn_epsilon = bn_layer.epsilon

    # 计算标准化的偏置加权平均值
    scale = bn_gamma / tf.sqrt(bn_moving_variance + bn_epsilon)
    shift = bn_beta - bn_moving_mean * scale

    # 融合卷积权重和BN参数后的新权重
    fused_weights = conv_weights * tf.reshape(scale, (1, 1, 1, -1))
    fused_bias = conv_bias * scale + shift

    return fused_weights, fused_bias

def getWeightsbn(model,saveweightdir, savetitle):
    '''
    Convert the model weights to JavaScript format for GEE.
    This function merges Conv2D and BatchNormalization layers, and saves the weights and biases in a JSON file.
    '''
    layers = []
    for layer in model.layers:
        if isinstance(layer, (Conv2D, Dense)):
            layers.append(layer)
        elif isinstance(layer, tf.keras.layers.BatchNormalization):
            prev_layer = layers.pop()
            # 合并卷积层和BN层，并替换卷积层权重
            fused_weights, fused_bias = fuse_conv_and_bn(prev_layer, layer)
            prev_layer.set_weights([fused_weights, fused_bias])
            layers.append(prev_layer)

    weights_data = [] 
    weights_list = []
    biases_list = []

    for layer in layers:
        weights, biases = layer.get_weights()
        weights_list.append(weights.tolist())  # 转换为嵌套列表
        biases_list.append(biases.tolist())

    # 保存为 JSON 文件
    with open(os.path.join(saveweightdir, 'model_weights.json'), 'w') as f:
        json.dump({'weights': weights_list, 'biases': biases_list}, f)
    
    # 转换权重和偏置
    weights_data = []
    for i, layer in enumerate(layers):
        weights, biases = layer.get_weights()
        
        # 转换卷积层的权重
        if isinstance(layer, Conv2D):
            # 转换轴顺序为 (filters, input_channels, kernel_height, kernel_width)
            weights = np.transpose(weights, (3, 2, 0, 1))
            # 转换为 JavaScript 可以直接使用的格式
            weights_js = 'var conv{}_weights = ee.List({});'.format(
                i + 1,
                json.dumps(weights.tolist())
            )
            
            biases_js = 'var conv{}_biases = ee.List({});'.format(
                i + 1,
                json.dumps(biases.tolist())
            )
        
            weights_data.append(weights_js)
            weights_data.append(biases_js)
        else:
            # 转换全连接层的权重
            weights_js = 'var fc{}_weights = ee.Array({});'.format(
                i + 1,
                json.dumps(weights.tolist())
            )
            
            biases_js = 'var fc{}_biases = ee.Array({});'.format(
                i + 1,
                json.dumps(biases.tolist())
            )
            
            weights_data.append(weights_js)
            weights_data.append(biases_js)

    saveweightdir1 = os.path.join(saveweightdir, f'{savetitle}.js')
    with open(saveweightdir1, 'w') as f:
        for wd in weights_data:
            f.write(wd + '\n')


def getWeights(model,saveweightdir,savetitle):
    '''
    Convert the model weights to JavaScript format for GEE.
    No batch normalization is used in this model.
    '''
    layers = [layer for layer in model.layers if isinstance(layer, (Conv2D, Dense))]

    weights_data = []  
    weights_list = []
    biases_list = []

    for layer in layers:
        weights, biases = layer.get_weights()
        weights_list.append(weights.tolist())  # 转换为嵌套列表
        biases_list.append(biases.tolist())

    # 保存为 JSON 文件
    with open('model_weights.json', 'w') as f:
        json.dump({'weights': weights_list, 'biases': biases_list}, f)
    # 转换权重和偏置
    weights_data = []

    for i, layer in enumerate(layers):
        weights, biases = layer.get_weights()
        
        # 转换卷积层的权重
        if isinstance(layer, (Conv2D)):
            # 转换轴顺序为 (filters, input_channels, kernel_height, kernel_width)
            weights = np.transpose(weights, (3, 2, 0, 1))
            # print('weights: {}'.format(weights.shape))
            # 转换为 JavaScript的格式
            weights_js = 'var conv{}_weights = ee.List({});'.format(
                i + 1,
                json.dumps(weights.tolist())
            )
            
            biases_js = 'var conv{}_biases = ee.List({});'.format(
                i + 1,
                json.dumps(biases.tolist())
            )
        
            weights_data.append(weights_js)
            weights_data.append(biases_js)
        else:
            # print('weights: {}'.format(weights.shape))
            # 转换全连接层的权重
            weights_js = 'var fc{}_weights = ee.Array({});'.format(
                i + 1,
                json.dumps(weights.tolist())
            )
            
            biases_js = 'var fc{}_biases = ee.Array({});'.format(
                i + 1,
                json.dumps(biases.tolist())
            )
            
            weights_data.append(weights_js)
            weights_data.append(biases_js)


    saveweightdir1=os.path.join(saveweightdir,f'{savetitle}.js')
    with open(saveweightdir1, 'w') as f:
        for wd in weights_data:
            f.write(wd + '\n')

## Model training and evualuation

In [ ]:
xtrain, ytrain, xval2010, yval2010, xval2022, yval2022,xvalall,yvalall=prepare_modelinput('../Data/traintestsamples.csv')

In [ ]:
# Define hyper parameters
numers_filters=[16,16,32,32,64]
channels='-'.join([str(x) for x in numers_filters])
filter=64 # number of filters in the fully connected  layer
spat=1 # spatial attention
bn=0 # batch normalization false
lr=0.0001 # Inital learning rate
batch_size=64
dropout=0.2
decayStep=20 # decay step for learning rate
epoch=100

def lr_schedule95(lr,decayStep,step_everyepoch):
        lr=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps=decayStep*step_everyepoch, 
        decay_rate=0.95,
        staircase=True)
        return lr

# Define save directory and file
savetitle=f'bnor{bn}epoch{epoch}_bn{batch_size}_spatt{spat}-channels{channels}-filter{filter}-decaystep{decayStep}-drop{dropout}'
savedir='../Land cover mapping/Model/'
resultsfile = os.path.join(savedir, f'{savetitle}_testmetrics.csv')
with open(resultsfile, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Define the table header, including indicators for different years
    headers = [ 
        'spat', 'lr', 'decay_step', 'batch_size', 'drop', 'channels', 'filter',
        'val_acc_total', 'f1_total', 'pctotal', 'uctotal', 
        'ptree_total', 'utree_total', 'f1tree_total', 'pgrass_total', 'ugr_total', 'f1grass_total',
        'phouse_total', 'uhouse_total', 'f1house_total','pbare_total', 'ubare_total', 'f1bare_total',
        
        'val_acc_2010', 'f1_2010', 'pc2010', 'uc2010',
        'ptree_2010', 'utree_2010', 'f1tree_2010', 'pgrass_2010','ugr_2010', 'f1grass_2010', 
        'phouse_2010', 'uhouse_2010', 'f1house_2010','pbare_2010', 'ubare_2010', 'f1bare_2010',
        'val_acc_2022', 'f1_2022', 'pc2022', 'uc2022',
        'ptree_2022', 'utree_2022', 'f1tree_2022', 'pgrass_2022','ugr_2022', 'f1grass_2022', 
        'phouse_2022', 'uhouse_2022', 'f1house_2022','pbare_2022', 'ubare_2022', 'f1bare_2022']
    
    csvwriter.writerow(headers) 
    # Create directories for saving results
    saveconfusionjpgdir =os.path.join(savedir, 'confusionjpg')
    os.makedirs(saveconfusionjpgdir, exist_ok=True)
    saveweightdir =os.path.join(savedir, 'weights')
    saveconfusiontrainjpgdir =os.path.join(savedir, 'confusiontrainjpg')
    os.makedirs(saveconfusiontrainjpgdir, exist_ok=True)
    os.makedirs(saveweightdir, exist_ok=True)
    savejpgdir =os.path.join(savedir, 'curve')
    os.makedirs(savejpgdir, exist_ok=True)
    savemodeldir =os.path.join(savedir, 'model')
    os.makedirs(savemodeldir, exist_ok=True)
    checkpoint_path = os.path.join(savemodeldir, f'{savetitle}.h5')
    saveweightpath=os.path.join(saveweightdir, f'{savetitle}.js')
    # learn rate schedule
    step_everyepoch = xtrain.shape[0] // batch_size
    lratio = lr_schedule95(lr, decayStep, step_everyepoch)
    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=lratio)
    model = hycnn2d(inputshape=(xtrain.shape[1], xtrain.shape[1], xtrain.shape[-1]), 
                    numers_filters=numers_filters, filter=filter, drop=dropout,spat=spat,bn=bn)
    model.compile(optimizer=optimizer, 
                loss=tf.keras.losses.CategoricalCrossentropy(), 
                metrics=['accuracy'])

    # Set up checkpoint
    checkpoint = ModelCheckpoint(checkpoint_path, monitor="val_accuracy",
                                mode="max", save_best_only=True)
    # Train model
    history = model.fit(xtrain, ytrain, epochs=epoch, batch_size=batch_size, 
                        callbacks=[checkpoint], validation_data=(xvalall, yvalall), verbose=0)
    
    # Evaluate model
    model.load_weights(checkpoint_path)

    # Collect metrics for overall validation set
    val_acc_total, kappa_total, f1_total, producer_accuracy_total, user_accuracy_total, ptree_total, \
    utree_total, f1tree_total, pgrass_total, ugr_total, f1grass_total, phouse_total, \
    uhouse_total, f1house_total, pb_total, ub_total, f1b_total,pwater,uwater,f1water = test_metrics(model, (xvalall, yvalall))
    # Collect metrics for 2010
    val_acc_2010, kappa_2010, f1_2010, producer_accuracy_2010, user_accuracy_2010, ptree_2010, \
    utree_2010, f1tree_2010, pgrass_2010, ugr_2010, f1grass_2010, phouse_2010, uhouse_2010, f1house_2010, \
    pb_2010, ub_2010, f1b_2010,pwater2010,uwater2010,f1water2010 = test_metrics(model, (xval2010, yval2010))

    # Collect metrics for 2022
    val_acc_2022, kappa_2022, f1_2022, producer_accuracy_2022, user_accuracy_2022, ptree_2022, \
    utree_2022, f1tree_2022, pgrass_2022, ugr_2022, f1grass_2022, phouse_2022, uhouse_2022, f1house_2022, \
    pb_2022, ub_2022, f1b_2022,pwater2022,uwater2022,f1water2022 = test_metrics(model, (xval2022, yval2022))

    # Prepare row with metrics from all years
    row = [
        spat, lr, decayStep, batch_size, dropout, channels, filter,
        val_acc_total, f1_total, producer_accuracy_total, user_accuracy_total, 
        ptree_total, utree_total, f1tree_total, pgrass_total, ugr_total, f1grass_total, 
        phouse_total, uhouse_total, f1house_total, pb_total, ub_total, f1b_total,
        val_acc_2010, f1_2010, producer_accuracy_2010, user_accuracy_2010, 
        ptree_2010, utree_2010, f1tree_2010, pgrass_2010, ugr_2010, f1grass_2010, 
        phouse_2010, uhouse_2010, f1house_2010,pb_2010, ub_2010, f1b_2010,
        
        
        val_acc_2022, f1_2022, producer_accuracy_2022, user_accuracy_2022, 
        ptree_2022, utree_2022, f1tree_2022, pgrass_2022, ugr_2022, f1grass_2022, 
        phouse_2022, uhouse_2022, f1house_2022,pb_2022, ub_2022, f1b_2022
    ]



    confusionmatrix(model,saveconfusionjpgdir,'bothyears',savetitle,xvalall,yvalall)
    
    confusionmatrix(model,saveconfusionjpgdir,2010,savetitle,xval2010,yval2010)

    confusionmatrix(model,saveconfusionjpgdir,2022,savetitle,xval2022,yval2022)

    plot_history(history, savejpgdir,savetitle)
    # Save model weights
    if bn:
        getWeightsbn(model,saveweightdir,savetitle)
    else:
        getWeights(model,saveweightdir,savetitle)
    csvwriter.writerow(row)
    csvfile.flush() 

